In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from scipy.sparse import csr_matrix
from sklearn.metrics import confusion_matrix
from IPython.display import Image
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.naive_bayes import BernoulliNB, ComplementNB, MultinomialNB, GaussianNB
from sklearn import metrics
from sklearn.utils import resample

In [ ]:
df = pd.read_csv("Cleaned_data_dummy_version_final.csv")
df.head()

In [ ]:
## Normalize
y = df["Price"]
X = df.drop(columns=["Price"])
scaler = MinMaxScaler()
X = pd.DataFrame(scaler.fit_transform(X),columns= X.columns)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=240)

In [ ]:
def create_cm(t1, t2):
    cm = confusion_matrix(t1, t2)
    plt.matshow(cm)
    plt.title('Confusion matrix')
    plt.colorbar()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.show()
X_train

In [ ]:
## Oversampling Minority
df["Price"].value_counts()

In [ ]:
#combine our training info back together for now
X =X_train
X['Price'] = y_train

# separate minority and majority classes
not_churn = X[X['Price']==1] #majority
churn = X[X['Price']==0] #minority
churn_2 = X[X['Price']==2] #minority

print(not_churn.shape)
print(churn.shape)
print(churn_2.shape)

In [ ]:
# upsample minority
churn_upsampled = resample(churn, replace=True, # sample with replacement
                           n_samples=len(not_churn), # match number in majority class
                           random_state=27) # reproducible results

churn_upsampled_2 = resample(churn_2, replace=True, # sample with replacement
                           n_samples=len(not_churn), # match number in majority class
                           random_state=27) # reproducible results

# combine majority and upsampled minority
upsampled = pd.concat([not_churn, churn_upsampled, churn_upsampled_2])

# check new class counts
upsampled['Price'].value_counts()

In [ ]:
# split our X and y back out
y_train_over = upsampled['Price']
X_train_over = upsampled.drop('Price', axis=1)

In [ ]:
## Undersample Majority
X =X_train
X['Price'] = y_train

not_churn = X[X['Price']==1]
churn = X[X['Price']==0]
churn_2 = X[X['Price']==2]
print(not_churn.shape)
print(churn.shape)
print(len(not_churn))

In [ ]:
not_churn_downsampled = resample(not_churn,
                                replace = False, # sample without replacement
                                #n_samples = len(churn), # match minority n
                                n_samples = len(churn), # create 2:1 ratio
                                random_state = 27) # reproducible results

# combine minority and downsampled majority
downsampled = pd.concat([not_churn_downsampled,churn_2, churn])

# checking counts
downsampled['Price'].value_counts()

In [ ]:
y_train_under = downsampled['Price']
X_train_under = downsampled.drop('Price', axis=1)
X_train_under

In [ ]:
##NB Model
###No imbalace process
X_train = X_train.drop(columns=['Price'])
model = MultinomialNB()
model.fit(X_train, y_train)

NBnews_predicted = model.predict(X_test)

print(model.score(X_test, y_test))
print(classification_report(y_test, NBnews_predicted))
create_cm(y_test, NBnews_predicted)


In [ ]:
### NB - Undersample
model = MultinomialNB()
model.fit(X_train_under, y_train_under)

NBnews_predicted = model.predict(X_test)

print(model.score(X_test, y_test))
print(classification_report(y_test, NBnews_predicted))
create_cm(y_test, NBnews_predicted)

In [ ]:
### NB - Oversample
model = MultinomialNB()
model.fit(X_train_over, y_train_over)

NBnews_predicted = model.predict(X_test)

print(model.score(X_test, y_test))
print(classification_report(y_test, NBnews_predicted))
create_cm(y_test, NBnews_predicted)